# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tbanerje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import pickle

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con = engine)
X = df['message']
Y = df.iloc[:,4:]

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    #sanitize - replace non-alphanumeric char with space, and normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #remove stop words
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    
    #lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = []
    for word in words:
        token = lemmatizer.lemmatize(word).lower().strip()
        tokens.append(token)
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 40)
pipeline.fit(X_train, Y_train)

D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change 

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...te=None, verbose=0,
            warm_start=False),
          n_jobs=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def display_results(y_test, y_pred, category_names):
    for i in range(len(category_names)):
        print("Category name:", category_names[i])
        print(classification_report(y_test.values[:, i], y_pred[:, i]))

In [9]:
Y_pred = pipeline.predict(X_test)
category_names = Y.columns

In [10]:
display_results(Y_test, Y_pred, category_names)

Category name: related
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      1246
           1       0.84      0.94      0.89      3968
           2       0.18      0.37      0.24        30

   micro avg       0.82      0.82      0.82      5244
   macro avg       0.58      0.58      0.56      5244
weighted avg       0.81      0.82      0.80      5244

Category name: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4324
           1       0.81      0.42      0.55       920

   micro avg       0.88      0.88      0.88      5244
   macro avg       0.85      0.70      0.74      5244
weighted avg       0.87      0.88      0.86      5244

Category name: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5223
           1       0.00      0.00      0.00        21

   micro avg       1.00      1.00      1.00      5244
   macr

D:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.86      0.95      0.91      3773
           1       0.84      0.61      0.71      1471

   micro avg       0.86      0.86      0.86      5244
   macro avg       0.85      0.78      0.81      5244
weighted avg       0.86      0.86      0.85      5244

Category name: floods
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4831
           1       0.82      0.24      0.37       413

   micro avg       0.94      0.94      0.94      5244
   macro avg       0.88      0.62      0.67      5244
weighted avg       0.93      0.94      0.92      5244

Category name: storm
              precision    recall  f1-score   support

           0       0.94      0.99      0.96      4735
           1       0.77      0.44      0.56       509

   micro avg       0.93      0.93      0.93      5244
   macro avg       0.86      0.71      0.76      5244
weighted avg       0.93      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'vectorizer__ngram_range': ((1, 1), (1, 2)),
        'tfidf__use_idf': [True, False]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(X_train, Y_train)
print('Fitted.')

D:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 50.8min finished
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\anaconda3\lib\site-packages\sklearn\ensemble\fores

Fitted.


In [14]:
Y_pred = cv.predict(X_test)

In [15]:
display_results(Y_test, Y_pred, category_names)

Category name: related
              precision    recall  f1-score   support

           0       0.69      0.44      0.54      1246
           1       0.84      0.93      0.88      3968
           2       0.31      0.40      0.35        30

   micro avg       0.81      0.81      0.81      5244
   macro avg       0.61      0.59      0.59      5244
weighted avg       0.80      0.81      0.80      5244

Category name: request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      4324
           1       0.82      0.43      0.57       920

   micro avg       0.88      0.88      0.88      5244
   macro avg       0.86      0.71      0.75      5244
weighted avg       0.88      0.88      0.87      5244

Category name: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5223
           1       0.00      0.00      0.00        21

   micro avg       1.00      1.00      1.00      5244
   macr

D:\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4831
           1       0.86      0.32      0.47       413

   micro avg       0.94      0.94      0.94      5244
   macro avg       0.90      0.66      0.72      5244
weighted avg       0.94      0.94      0.93      5244

Category name: storm
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4735
           1       0.76      0.29      0.42       509

   micro avg       0.92      0.92      0.92      5244
   macro avg       0.85      0.64      0.69      5244
weighted avg       0.91      0.92      0.91      5244

Category name: fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5174
           1       0.00      0.00      0.00        70

   micro avg       0.99      0.99      0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.97      0.99

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
pickle.dump(cv, open('disaster_resp_classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.